# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 06**: Data pipeline with Neo4j

**Date**: October 5nd 2025

**Student Name**: José Alfredo Calvillo Gómez

**Professor**: Pablo Camarillo Ramirez

# Dataset description

El dataset utilizado proviene de **Kaggle** y contiene relaciones entre usuarios de una red social. El archivo **facebook_combined.txt** está compuesto por dos columnas: **source** y **destination**, que representan las relaciones de amistad entre los usuarios. 

- **Nodos**: Son los usuarios de la red social (representados en las columnas **source** y **destination**).
- **Aristas**: Son las relaciones de amistad entre los usuarios. Cada fila indica que el usuario en la columna **source** es amigo del usuario en la columna **destination**.

Este dataset es adecuado para realizar un análisis de grafos, donde se pueden identificar patrones de conexiones, comunidades y otras características de las relaciones entre los usuarios.


# Data ingestion

In [ ]:
# Data Ingestion

# Inicialización de findspark para encontrar Spark
import findspark
findspark.init()

# Importar la librería necesaria para PySpark
from pyspark.sql import SparkSession

# Iniciar la sesión de Spark
spark = SparkSession.builder \
    .appName("Examples on SparkSQL") \
    .master("spark://spark-master:7077") \  # Asegúrate de que esto esté correcto
    .config("spark.ui.port", "4040") \
    .getOrCreate()

# Establecer el contexto de Spark
sc = spark.sparkContext
sc.setLogLevel("ERROR")

# Leer el archivo facebook_combined.txt
file_path = '/mnt/data/ego-Facebook/facebook_combined.txt'

# Cargar el archivo como un DataFrame de PySpark
edges_df = spark.read.text(file_path)

# Mostrar las primeras filas para entender la estructura
edges_df.show(5)


In [ ]:
# Build schema
# Import your module


# Transformations

In [ ]:
from pyspark.sql.functions import split

# Separar las columnas de origen y destino
edges_df = edges_df.withColumn('src', split(edges_df['value'], ' ').getItem(0)) \
                   .withColumn('dst', split(edges_df['value'], ' ').getItem(1)) \
                   .select('src', 'dst')

# Mostrar las primeras filas después de la transformación
edges_df.show(5)

# Crear un DataFrame de nodos a partir de las columnas src y dst
nodes_df = edges_df.select('src').union(edges_df.select('dst')).distinct()

# Mostrar los primeros nodos
nodes_df.show(5)


# Writing Data in Neo4j

In [ ]:
# Writing Data in Neo4j

# Escribir nodos en Neo4j
nodes_df.write \
    .format("neo4j") \
    .option("url", "bolt://localhost:7687") \
    .option("authentication.basic.username", "neo4j") \
    .option("authentication.basic.password", "neo4j@1234") \
    .mode("overwrite") \
    .option("node.label", "User") \
    .save()

# Escribir aristas en Neo4j
edges_df.write \
    .format("neo4j") \
    .option("url", "bolt://localhost:7687") \
    .option("authentication.basic.username", "neo4j") \
    .option("authentication.basic.password", "neo4j@1234") \
    .mode("overwrite") \
    .option("relationship.type", "FRIEND") \
    .option("relationship.source.start", "src") \
    .option("relationship.target.end", "dst") \
    .save()


# Read and Query Graphs with PySpark

In [ ]:
# Querying the graph

query = "MATCH (u:User)-[r:FRIEND]->(v:User) RETURN u.name, v.name LIMIT 5"

# Leer desde Neo4j
result_df = spark.read \
    .format("neo4j") \
    .option("url", "bolt://localhost:7687") \
    .option("authentication.basic.username", "neo4j") \
    .option("authentication.basic.password", "neo4j@1234") \
    .option("query", query) \
    .load()

result_df.show()


In [ ]:
sc.stop()